In [4]:
import pandas as pd
import os
import datetime
from tqdm import tqdm
# from sen_hub import get_all_bands_response
import joblib
from pyproj import CRS

ModuleNotFoundError: ignored

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%load '/content/drive/MyDrive/DataQ/Business development/Products/Jareed/Class/training pipeline/sen_hub.py'

In [6]:
!pip install sentinelhub
!pip install geopandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.4/244.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.2 MB/s eta 0:00:00
  Created wheel for utm: filename=utm-0.7.0-py3-none-any.whl size=6085 sha256=5d5748bb042f1c341c3d79bc674e43b9e00ad7ac00647a5978d8b883ceacfb6e
  Stored in directory: /root/.cache/pip/wheels/2f/a1/c8/543df0e8f5e824c3e92a432e32deb9cd89ae686095ee8cfcbe
Successfully built utm


In [7]:
import geopandas as gpd


In [10]:
%%time
with open('/content/drive/MyDrive/DataQ/Business development/Products/Jareed/Class/crs.txt', 'r') as file:
    crs = CRS.from_wkt(file.read())


CPU times: user 9.31 ms, sys: 2.85 ms, total: 12.2 ms
Wall time: 340 ms


In [11]:
#load the labels file [year, geometry, Crop_Type, Rainfed, fieldID]

gdf=joblib.load('/content/drive/MyDrive/DataQ/Business development/Products/Jareed/Class/training pipeline/labels.joblib')

FileNotFoundError: ignored

In [ ]:
gdf

,Year,geometry,Crop_Type,State,Rainfed,fieldID
0,2021,"POLYGON ((13.04430 32.80291, 13.04636 32.80190...",Sorghum,WhiteNileDC,1,WN1
1,2021,"POLYGON ((13.04430 32.80291, 13.04636 32.80190...",Sorghum,WhiteNileDC,1,WN2
2,2021,"POLYGON ((13.04430 32.80291, 13.04636 32.80190...",Sorghum,WhiteNileDC,1,WN3
3,2021,"POLYGON ((13.04430 32.80291, 13.04636 32.80190...",Sorghum,WhiteNileDC,1,WN4
4,2021,"POLYGON ((13.04430 32.80291, 13.04636 32.80190...",Sorghum,WhiteNileDC,1,WN5
...,...,...,...,...,...,...
84,2022,"POLYGON ((13.04430 32.80291, 13.04636 32.80190...",Cotton,BlueNileDC,1,BNDC21
85,2022,"POLYGON ((13.04430 32.80291, 13.04636 32.80190...",Cotton,BlueNileDC,1,BNDC22
86,2022,"POLYGON ((13.04430 32.80291, 13.04636 32.80190...",Sorghum,BlueNileDC,1,BNDC23
87,2022,"POLYGON ((13.04430 32.80291, 13.04636 32.80190...",Cotton,BlueNileDC,1,BNDC24


In [ ]:
years = [2020,2021,2022] #set target years

In [ ]:
# download the bands configured in the senhub config file for each field / Date pairing

gdf['bbox'] = [[r.minx, r.miny, r.maxx, r.maxy] for _,r in gdf.bounds.iterrows()]
#download images for all years available
for year in years:
    capture_dates = [
             ( datetime.datetime(year, 7 , 10).strftime("%Y-%m-%d"), datetime.datetime(year, 7 , 20).strftime("%Y-%m-%d") ),
             ( datetime.datetime(year, 8 , 10).strftime("%Y-%m-%d"), datetime.datetime(year, 8 , 20).strftime("%Y-%m-%d") ),
             #( datetime.datetime(year, 9 , 1).strftime("%Y-%m-%d"), datetime.datetime(year, 9 , 5).strftime("%Y-%m-%d") ),
             ( datetime.datetime(year, 9 , 10).strftime("%Y-%m-%d"), datetime.datetime(year, 9 , 20).strftime("%Y-%m-%d") ),
             #( datetime.datetime(year, 9 , 25).strftime("%Y-%m-%d"), datetime.datetime(year, 9 , 30).strftime("%Y-%m-%d") ),
             #( datetime.datetime(year, 10 , 1).strftime("%Y-%m-%d"), datetime.datetime(year, 10 , 5).strftime("%Y-%m-%d") ),
             ( datetime.datetime(year, 10 , 10).strftime("%Y-%m-%d"), datetime.datetime(year, 10 , 20).strftime("%Y-%m-%d") ),
             #( datetime.datetime(year, 10 , 25).strftime("%Y-%m-%d"), datetime.datetime(year, 10 , 30).strftime("%Y-%m-%d") ),
             #( datetime.datetime(year, 11 , 1).strftime("%Y-%m-%d"), datetime.datetime(year, 11 , 5).strftime("%Y-%m-%d") ),
             ( datetime.datetime(year, 11 , 10).strftime("%Y-%m-%d"), datetime.datetime(year, 11 , 20).strftime("%Y-%m-%d") ),
             #( datetime.datetime(year, 11 , 25).strftime("%Y-%m-%d"), datetime.datetime(year, 11 , 30).strftime("%Y-%m-%d") ),
             ( datetime.datetime(year, 12, 10).strftime("%Y-%m-%d"), datetime.datetime(year, 12, 20).strftime("%Y-%m-%d") )]
    #Download images for all dates (7 (one each month for each field)
    for start_date, end_date in capture_dates:
        output_dir = f'./new_downloads/{year}/{start_date}/'
        if not os.path.exists(output_dir) : os.makedirs(output_dir)
        for i,r in tqdm(gdf.iterrows()):
            if r.Year!=year:
                continue
            output_dir_img = output_dir + f'Field_Id_{r.fieldID}'
            #skip download if the image is downloaded
            if  os.path.exists(output_dir_img):
                print(f'The Image {output_dir_img} is there ')
                continue

            # get configured bands for each field and dat pairing
            img = get_all_bands_response(bbox=r.bbox, start_date=start_date, end_date=end_date,output_dir=output_dir_img, save_data=True)
        #break

89it [00:16,  5.50it/s]
89it [00:24,  3.68it/s]
89it [00:22,  4.02it/s]
89it [00:18,  4.71it/s]
89it [00:15,  5.84it/s]
89it [00:19,  4.48it/s]
89it [00:19,  4.54it/s]
89it [00:15,  5.76it/s]
89it [00:14,  6.16it/s]
89it [00:21,  4.21it/s]
89it [00:15,  5.71it/s]
89it [00:18,  4.75it/s]
89it [00:53,  1.67it/s]
89it [01:04,  1.37it/s]
89it [00:59,  1.49it/s]
89it [00:56,  1.58it/s]
89it [00:45,  1.94it/s]
89it [00:59,  1.50it/s]
89it [01:17,  1.14it/s]
89it [00:56,  1.57it/s]
89it [00:56,  1.57it/s]
89it [00:53,  1.65it/s]
89it [00:56,  1.57it/s]
89it [01:07,  1.32it/s]


In [ ]:
#update the labels gdf by adding image paths to each field.
pathS=[]
for i, x in gdf.iterrows():
    for year in years:
        if x.Year!=year:
                continue
        capture_dates = [
             ( datetime.datetime(year, 7 , 10).strftime("%Y-%m-%d"), datetime.datetime(year, 7 , 20).strftime("%Y-%m-%d") ),
             ( datetime.datetime(year, 8 , 10).strftime("%Y-%m-%d"), datetime.datetime(year, 8 , 20).strftime("%Y-%m-%d") ),
             ( datetime.datetime(year, 9 , 1).strftime("%Y-%m-%d"), datetime.datetime(year, 9 , 5).strftime("%Y-%m-%d") ),
             ( datetime.datetime(year, 9 , 10).strftime("%Y-%m-%d"), datetime.datetime(year, 9 , 20).strftime("%Y-%m-%d") ),
             ( datetime.datetime(year, 9 , 25).strftime("%Y-%m-%d"), datetime.datetime(year, 9 , 30).strftime("%Y-%m-%d") ),
             ( datetime.datetime(year, 10 , 1).strftime("%Y-%m-%d"), datetime.datetime(year, 10 , 5).strftime("%Y-%m-%d") ),
             ( datetime.datetime(year, 10 , 10).strftime("%Y-%m-%d"), datetime.datetime(year, 10 , 20).strftime("%Y-%m-%d") ),
             ( datetime.datetime(year, 10 , 25).strftime("%Y-%m-%d"), datetime.datetime(year, 10 , 30).strftime("%Y-%m-%d") ),
             ( datetime.datetime(year, 11 , 1).strftime("%Y-%m-%d"), datetime.datetime(year, 11 , 5).strftime("%Y-%m-%d") ),
             ( datetime.datetime(year, 11 , 10).strftime("%Y-%m-%d"), datetime.datetime(year, 11 , 20).strftime("%Y-%m-%d") ),
             ( datetime.datetime(year, 11 , 25).strftime("%Y-%m-%d"), datetime.datetime(year, 11 , 30).strftime("%Y-%m-%d") ),
             ( datetime.datetime(year, 12, 10).strftime("%Y-%m-%d"), datetime.datetime(year, 12, 20).strftime("%Y-%m-%d") )]
        paths=[]
        for start_date, end_date in capture_dates:
            path=f'./new_downloads/{year}/{start_date}'
            imagePath=f'{path}/Field_Id_{x.fieldID}'
            imagePath=f'./samer_downloader/{imagePath}/{os.listdir(imagePath+"/")[0]}/response.tiff'
            paths.append(imagePath)
        pathS.append(paths)

gdf['imgPaths']=pathS

/home/samersurag/mambaforge/envs/dataq-rep/lib/python3.9/site-packages/geopandas/geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [ ]:
gdf

,State,Year,Crop_Type,Rainfed,fieldID,geometry,imgPaths
0,BlueNile1,2020,Sorghum,1,0_BN_1,"POLYGON ((33.89870 11.89241, 33.90450 11.88516...",[./samer_downloader/./new_downloads/2020/2020-...
1,BlueNile1,2020,Cotton,1,1_BN_1,"POLYGON ((33.86977 11.87022, 33.87552 11.86313...",[./samer_downloader/./new_downloads/2020/2020-...
2,BlueNile1,2020,Cotton,1,2_BN_1,"POLYGON ((33.89154 11.88682, 33.89739 11.87958...",[./samer_downloader/./new_downloads/2020/2020-...
3,BlueNile1,2020,Sorghum,1,3_BN_1,"POLYGON ((33.89284 11.84132, 33.89842 11.83426...",[./samer_downloader/./new_downloads/2020/2020-...
4,BlueNile1,2020,Sorghum,1,4_BN_1,"POLYGON ((33.88862 11.86126, 33.88290 11.86856...",[./samer_downloader/./new_downloads/2020/2020-...
5,BlueNile1,2020,Sorghum,1,5_BN_1,"POLYGON ((33.91439 11.85807, 33.92008 11.85101...",[./samer_downloader/./new_downloads/2020/2020-...
6,BlueNile1,2020,Cotton,1,6_BN_1,"POLYGON ((33.91023 11.87795, 33.90450 11.88516...",[./samer_downloader/./new_downloads/2020/2020-...
7,BlueNile1,2020,Cotton,1,7_BN_1,"POLYGON ((33.92165 11.86367, 33.92734 11.85660...",[./samer_downloader/./new_downloads/2020/2020-...
8,BlueNile1,2020,Sorghum,1,8_BN_1,"POLYGON ((33.91746 11.88357, 33.91171 11.89075...",[./samer_downloader/./new_downloads/2020/2020-...
9,BlueNile1,2020,Cotton,1,9_BN_1,"POLYGON ((33.89447 11.91247, 33.88874 11.91959...",[./samer_downloader/./new_downloads/2020/2020-...


In [ ]:
joblib.dump(gdf, 'farmsDClabels.joblib')

['farmsDClabels.joblib']